Realizando importações necessárias para a implementação do Iceberg

In [ ]:
from iceberg.api import Schema
from iceberg.api.types import Types
from iceberg.hive import HiveCatalog
from iceberg import FileFormat

: 

Criando a tabela Iceberg

In [ ]:
schema = Schema([
    Types.NestedField.required(1, "dt", Types.DateType.get()),
    Types.NestedField.required(2, "AverageTemperature", Types.DoubleType.get()),
    Types.NestedField.required(3, "AverageTemperatureUncertainty", Types.DoubleType.get()),
    Types.NestedField.required(4, "City", Types.StringType.get()),
    Types.NestedField.required(5, "Country", Types.StringType.get()),
    Types.NestedField.required(6, "Latitude", Types.StringType.get()),
    Types.NestedField.required(7, "Longitude", Types.StringType.get())
])

catalog = HiveCatalog.builder().uri("hive://<sua-uri>").build()

table = catalog.create_table(schema).location("s3://<seu-bucket>/<seu-caminho>").format(FileFormat.PARQUET).build()

data = [
    {
        "dt": "2024-04-21", 
        "AverageTemperature": 10, 
        "AverageTemperatureUncertainty": 10,
        "City": "Porto Alegre",
        "Country": "Brazil",
        "Latitude": "0N",
        "Longitude": "0E"
    }
]

table.new_append().append_file_data(data).commit()

Consultando os dados da tabela

In [ ]:
snapshot = table.new_transaction().current_snapshot()
rows = table.new_scan(). \
use_snapshot(snapshot.id()). \
select("dt", "AverageTemperature", "City").plan_scan().execute()

for row in rows:
    print(row)